In [33]:
import urllib3
import certifi
import bs4
import pandas as pd
from datetime import datetime

In [31]:
url = 'https://www.worldometers.info/coronavirus/'


http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
response = http.request('GET', url)

data = response.data.decode('utf-8')
soup = bs4.BeautifulSoup(data)

table_data = soup.find(id="main_table_countries_today")

for row in table_data.findAll('tr'):
    for column in row.findAll('td'):
        column.string = column.get_text().replace(',', '').replace('+','')

current_data = pd.read_html(str(table_data))
needed_columns = ['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths']
needed_data = current_data[0][needed_columns]
needed_data[['TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths']] = needed_data[['TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths']].apply(pd.to_numeric)
needed_data.fillna(0, inplace=True)

In [36]:
needed_data.to_excel("./Data/WorldData.xlsx", sheet_name = datetime.today().strftime('%Y-%m-%d'))  
